In [ ]:
import pandas as pd

from summer2 import CompartmentalModel
from summer2.parameters import Parameter

In [ ]:
demog_data = pd.read_csv(
    "./csvs/model_3_2.csv",
    header=53,
    index_col=0,
    nrows=36501,
    usecols=range(5),
)

In [ ]:
config = {
    "tot_popn": 1e5,
    "infectious_seed": 1.,
    "end_time": 100.,
}
compartments = ("Susceptible", "Pre-infectious", "Infectious", "Immune")
model = CompartmentalModel(
    times=(0, config["end_time"] * 365.),
    compartments=compartments,
    infectious_compartments=["Infectious"],
)
model.set_initial_population(
    distribution={
        "Susceptible": config["tot_popn"] - config["infectious_seed"],
        "Infectious": config["infectious_seed"],
    }
)
ave_infous = Parameter("ave_infous")
model.add_infection_frequency_flow(
    name="infection", 
    contact_rate=Parameter("r0") / ave_infous,
    source="Susceptible", 
    dest="Pre-infectious"
)
model.add_transition_flow(
    name="progression", 
    fractional_rate=1. / Parameter("ave_preinfous"),
    source="Pre-infectious", 
    dest="Infectious"
)
model.add_transition_flow(
    name="recovery", 
    fractional_rate=1. / ave_infous,
    source="Infectious", 
    dest="Immune"
)
model.add_universal_death_flows(
    "universal_death",
    death_rate=1. / Parameter("life_expectancy") / 365.
)
model.add_replacement_birth_flow(
    "births",
    "Susceptible"
)
parameters = {
    "r0": 13.,
    "ave_preinfous": 8.,
    "ave_infous": 7.,
    "life_expectancy": 70.,
}
model.run(parameters=parameters, solver="euler")
demog_values = model.get_outputs_df()
assert abs(demog_data - demog_values).max().max() < 1e5